In [1]:
from confluent_kafka import Consumer, KafkaError
from confluent_kafka.schema_registry import SchemaRegistryClient
from confluent_kafka.schema_registry.protobuf import ProtobufDeserializer
from confluent_kafka.serialization import SerializationContext, MessageField
import my_event_pb2
import math

In [2]:
def calculate_sqrt(value):
    return math.sqrt(value)

In [3]:
schema_registry_conf = {'url': 'http://schema-registry:8081'}
schema_registry_client = SchemaRegistryClient(schema_registry_conf)

deserializer_conf = {'use.deprecated.format': False}
protobuf_deserializer = ProtobufDeserializer(my_event_pb2.MyEvent, deserializer_conf)

/opt/conda/lib/python3.11/site-packages/confluent_kafka/schema_registry/protobuf.py:513: UserWarning: MessageFactory class is deprecated. Please use GetMessageClass() instead of MessageFactory.GetPrototype. MessageFactory class will be removed after 2024.
  self._msg_class = MessageFactory().GetPrototype(descriptor)


In [4]:
c = Consumer({
    'bootstrap.servers': 'kafka-1:29092,kafka-2:29093,kafka-3:29094',
    'group.id': 'mygroup',
    'auto.offset.reset': 'earliest'
})

c.subscribe(['my-topic'])

In [5]:
while True:
    msg = c.poll(5.0)
    if msg is None:
        continue
    if msg.error():
        if msg.error().code() == KafkaError._PARTITION_EOF:
            continue
        else:
            print(msg.error())
            break

    event = protobuf_deserializer(msg.value(), SerializationContext(msg.topic(), MessageField.VALUE))
    
    original_value = event.value
    calculated_value = calculate_sqrt(original_value)

    print(f'Received event with timestamp: {event.timestamp}')
    print(f'Square Root Consumer - Number: {original_value}, Square Root: {calculated_value:.2f}\n')

c.close()

Received event with timestamp: seconds: 1725445696
nanos: 131127000

Square Root Consumer - Number: 72, Square Root: 8.49

Received event with timestamp: seconds: 1725445726
nanos: 140629000

Square Root Consumer - Number: 57, Square Root: 7.55

Received event with timestamp: seconds: 1725445686
nanos: 117672000

Square Root Consumer - Number: 47, Square Root: 6.86

Received event with timestamp: seconds: 1725445681
nanos: 104474000

Square Root Consumer - Number: 87, Square Root: 9.33

Received event with timestamp: seconds: 1725445731
nanos: 141542000

Square Root Consumer - Number: 96, Square Root: 9.80

Received event with timestamp: seconds: 1725445771
nanos: 156304000

Square Root Consumer - Number: 73, Square Root: 8.54



KeyboardInterrupt: 